In [8]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By

In [10]:
main_link = 'https://www.airbnb.com/s/New-York--NY--United-States/homes'
items = '?items_offset='
listings = []
types_list = []
ratings_list = []
price_list = []
for i in range(0, 6000, 20):
    try:
        link = main_link + items + str(i)
        print(link)
        driver = None
        driver = webdriver.Chrome('/usr/local/bin/chromedriver')
        driver.get(link)
        time.sleep(5)
        curr_listings = driver.find_elements(By.CLASS_NAME, '_1whrsux9')
        type_list = driver.find_elements(By.CLASS_NAME, '_3c0zz1')
        star_list = driver.find_elements(By.CLASS_NAME, '_10fy1f8')
        cost_list = driver.find_elements(By.CLASS_NAME, '_tyxjp1')
        if len(curr_listings) > 0 and len(type_list) > 0:
            listings += [place.text for place in curr_listings]
            types_list += [types.text for types in type_list]
            ratings_list += [rating.text for rating in star_list]
            price_list += [price.text for price in cost_list]
    except Exception as e:
        pass
print(listings, types_list)

https://www.airbnb.com/s/New-York--NY--United-States/homes?items_offset=0


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  if sys.path[0] == '':


https://www.airbnb.com/s/New-York--NY--United-States/homes?items_offset=20
https://www.airbnb.com/s/New-York--NY--United-States/homes?items_offset=40
["Hell's Kitchen Enclave: a Diamond in the Rough.", 'DOUBLE SHARED BATHROOM', 'Midtown West Hotel NYC', 'DOUBLE SHARED BATHROOM', 'Charming Astoria studio on quiet tree-lined street', 'SSB 9 - SINGLE SHARED BATHROOM', 'Bnb(30 day minimum)', 'Studio located in Upper East Side', '1 bedroom apt, 1 subway stop from Midtown', 'CONTACT BEFORE YOU BOOK/near Times Square', 'Sunny Room Brand New Apartment Close to Midtown', '240Nest #4 with view on the Empire State Building', "Murray Hill 'Light & Sweet'", 'Charming Studio in heart of the Upper East Side', 'Studio Loft: Cleaning CDC guidelines implemented', 'C☆zy Studio in Union Square Park', 'Nini’s Cozy Room', '<Compact room room>SofaBed + loft bed in Manhattan', 'GREAT Bright Manhtattan Apartment!', 'Nice, clean, cozy 1 bed apt in Kensington Brooklyn', '<Compact room room>SofaBed + loft bed in 

In [11]:
print(price_list, ratings_list)

['$68', '$55', '$75', '$61', '$50', '$55', '$45', '$100', '$106', '$39', '$56', '$99', '$126', '$102', '$99', '$87', '$55', '$85', '$99', '$45', '$85', '$63', '$75', '$55', '$85', '$83', '$121', '$57', '$121', '$69', '$130', '$110', '$99', '$179', '$59', '$62', '$43', '$91', '$100', '$125', '$135', '$116', '$61', '$61', '$88', '$86', '$62', '$49', '$47', '$89', '$58', '$85', '$95', '$50', '$100', '$99', '$170', '$72', '$63', '$148'] ['4.59', '5.0', '4.67', '5.0', '4.82', '5.0', '4.43', '5.0', '4.88', '5.0', '4.68', '4.91', '4.75', '5.0', '4.75', '4.29', '4.88', '4.0', '4.82', '4.80', '4.69', '5.0', '4.48', '4.56', '4.95', '4.68', '5.0', '5.0', '4.79', '4.78', '4.72', '4.68', '5.0', '4.89', '4.85', '4.75', '4.74', '4.71', '4.84', '5.0', '4.94', '4.46', '4.88', '4.82', '4.60', '4.86', '4.78', '4.56']
